In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpmath
import dautil as dl
from IPython.display import HTML

In [ ]:
def model(t):
    mu, C, w, phi = (9.6848106, -7.59870042, -0.01766333, -5.83349705)

    return mu + C * mpmath.cos(w * t + phi)

In [ ]:
def diff_model(t):
    return mpmath.diff(model, t)

In [ ]:
vals = dl.data.Weather.load()['TEMP'].dropna()
vals = dl.ts.groupby_yday(vals).mean()
diff_root = mpmath.findroot(diff_model, (1, 366), solver='anderson')

In [ ]:
days = range(1, 367)
poly = mpmath.chebyfit(model, (1, 366), 3)
poly = np.array([float(c) for c in poly])

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
context = dl.nb.Context('mpmath_fit')
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 1, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 1, context)
cp = dl.plotting.CyclePlotter(sp.ax)
cp.plot(days, [model(i) for i in days], label='Model')
cp.plot(days, vals, label='Data')
sp.ax.annotate(s='Root of derivative', xy=(diff_root, vals.max() - 1),
            xytext=(diff_root, vals.max() - 8),
            arrowprops=dict(arrowstyle='->'))
yvar = dl.data.Weather.get_header('TEMP')
sp.label(ylabel_params=yvar)

cp = dl.plotting.CyclePlotter(sp.next_ax())
cp.plot(days, vals, label='Data')
cp.plot(days, np.polyval(poly, days), label='Approximation')
sp.label(ylabel_params=yvar)
plt.tight_layout()
HTML(dl.report.HTMLBuilder().watermark())